In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import time
import h5py
import copy
import datetime
import ta
from sklearn.model_selection import train_test_split
# Do you wanna see?
verbose = True

import sys
sys.path.append('../')

from utils.data_extraction import load_data
from utils.data_cleaning import HFDataCleaning
from utils.generate_features import candleCreateNP_vect_final, generateFeatures_final
from utils.preprocessing_features_and_labels import extract_labels, align_features_and_labels

### Load data etc

In [2]:
# print(os.listdir())
try:
    path = 'a:/taqhdf5'  #'a:/taqhdf5'
    os.listdir(path)
except:
    path = 't:/taqhdf5'  #'a:/taqhdf5'
    os.listdir(path)    
# allFiles = os.listdir(path)
# print(len(allFiles), allFiles[:5], allFiles[-5:])
# print(allFiles[-10:])

#dates = np.array(['2020040' + str(i) if i < 10 else '202004' + str(i) for i in np.arange(1,32)]).astype(int)
dates = np.array(['20200401','20200402']).astype(int)

# Provide a list of tickers of interest
tickers = ['GOOG']#'MSFT'

# Do we need data on trades, quotes or both?
dataNeeded = 'trades' # 'trades', 'quotes' or 'both'

# run load_data()
if dataNeeded == 'trades':
    tradeData = load_data(dates, tickers, dataNeeded, path, verbose)
elif dataNeeded == 'quotes':
    quoteData = load_data(dates, tickers, dataNeeded, path, verbose)
elif dataNeeded == 'both':
    tradeData, quoteData = load_data(dates, tickers, dataNeeded, path, verbose)

['.ipynb_checkpoints', 'preprocessing.ipynb', 'testing_and_comparing_candlesticks.ipynb', 'testing_features.ipynb', 'testing_preprocessing_features_and_labels.ipynb']
8515 ['taq_19930315.h5', 'taq_19930104.h5', 'taq_19930317.h5', 'taq_19930105.h5', 'taq_19930316.h5'] ['taqquote_20200522.h5', 'taqquote_20200526.h5', 'taqquote_20200527.h5', 'taqquote_20200528.h5', 'taqquote_20200529.h5']
['taqquote_20200515.h5', 'taqquote_20200518.h5', 'taqquote_20200519.h5', 'taqquote_20200520.h5', 'taqquote_20200521.h5', 'taqquote_20200522.h5', 'taqquote_20200526.h5', 'taqquote_20200527.h5', 'taqquote_20200528.h5', 'taqquote_20200529.h5']
##### Date range #####

Date, Min: 20200401
Date, Max: 20200402

##### Data Extraction begins #####

trade data is being extracted..

### Trade Data ###

The raw H5 trade file contains:  ['TradeIndex', 'Trades'] 

Ticker Information:  (b'GOOG            ', 26900500, 71427) 

Sneak peak of the data

            utcsec ex  cond  volume    price TradeStopStockIndicator c

In [3]:
#cleanedData = HFDataCleaning(['P1','p2','t1','p3'],tradeData,'trade',['q'])


DATA_SAMPLE = 'full' # or 'stable'

if DATA_SAMPLE == 'stable':
    # P1 is used for keeping data within [9.5, 16]
    cleanedData = HFDataCleaning(['P1','p2','t1','p3'],tradeData,'trade',['q','d'])
elif DATA_SAMPLE == 'full':
    # P1_2 is used for keeping data within [9, 16.5]
    cleanedData = HFDataCleaning(['P1_2','p2', 't1', 'p3'],tradeData,'trade',['q','d'])

### Original

In [4]:
# # Aggregate data in candle sticks
# candleNP = candleCreateNP()

# candleNP.shape

In [5]:
# test_candles=candleCreateNP_vect()
# test_candles

In [6]:
candles = candleCreateNP_vect_final(cleanedData
                         ,1)

# candles_test = createCandles_test(cleanedData
#                                      ,1)

candles

array([[1122.26  , 1127.88  , 1119.51  , 1125.92  ],
       [1127.    , 1128.1099, 1120.    , 1124.0805],
       [1127.19  , 1128.08  , 1123.    , 1127.79  ],
       ...,
       [1121.1337, 1123.    , 1119.1834, 1120.99  ],
       [1122.58  , 1123.6   , 1121.18  , 1122.725 ],
       [1122.725 , 1124.    , 1119.21  , 1121.74  ]])

In [7]:
# lets not transpose :D
candles.T[0] == candles[:,0]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [8]:
# %timeit candles1 = createCandles(cleanedData,1)
# %timeit candles2 = createCandles_test(cleanedData,1)
# # No difference

# Implementing technical features

A library: https://technical-analysis-library-in-python.readthedocs.io/en/latest/

### Features used in the literature:

* Stochastic K - Implemented
* Stochastic D - Implemented
* Slow Stochastic D - Implemented
* Momentum - Same as difference
* ROC - Implemented
* Williams % R - Implemented
* A/D Oscillator
* Disparity 5 - Implemented
* Disparity 10 - Implemented
* Price Oscillator - (detrended) - Implemented
* Commodity Channel Index - Implemented
* RSI - Impliemented

Formulas: https://papers.ssrn.com/sol3/papers.cfm?abstract_id=876544

* Moving Average - Implemented
* Bias
* Exponential Moving Average - Implemented
* Difference - Same af Momentum
* True Range - (Average) - Implemented
* 

Formulas: https://www.sciencedirect.com/science/article/pii/S0957417407001819?via%3Dihub

#### Other Technical Features
* Moving Average Convergence Divergence (MACD) - Implemented

**Non-classical technical features**

* Bid/Ask prices of top of book
* Spread and mid price based on top og book
* Price derivatives

Formulas: https://www.tandfonline.com/doi/full/10.1080/14697688.2015.1032546?instName=UCL+%28University+College+London%29

In [12]:
n_feature_lags = 5
features = generateFeatures_final(data = candles, 
                                  listOfFeatures = [
#                                                     'pastobs',
                                                    'stok',
#                                                     'stod',
#                                                     'sstod',
#                                                     'wilr',
#                                                     'roc',
                                                    'rsi',
#                                                     'atr',
#                                                     'cci',
#                                                     'dpo',
#                                                     'sma',
#                                                     'ema',
#                                                     'macd',
#                                                     'dis5',
#                                                     'dis10',
                                                   ], 
                                   feature_lags = n_feature_lags) # 0 for no lags, 1 for 1 lag (returns both raw & first lag)
features 

,stok_lag0,rsi_lag0,stok_lag1,rsi_lag1,stok_lag2,rsi_lag2,stok_lag3,rsi_lag3,stok_lag4,rsi_lag4,stok_lag5,rsi_lag5
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,75.088920,50.096776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,66.831334,47.132255,75.088920,50.096776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Labels (drafts)

In [14]:
# def extract_labels(data = '', classes = 5, group_style = 'equal'):
    
#    # returns = ((data.T[-1][1:]/data.T[-1][0:-1])-1)*100
#     returns = ((data[1:, -1] / data[:-1, -1])-1)*100
    
#     labels = np.zeros(returns.shape[0])

#     if group_style == 'equal':
#         thresholdsMin = [np.array_split(np.sort(returns),classes)[i].min() for i in np.arange(classes)]
#         thresholdsMax = [np.array_split(np.sort(returns),classes)[i].max() for i in np.arange(classes)]    
#     elif group_style != 'equal':
#         raise ValueError(f'group_style {group_style} not implemented')
    
#     for i in np.arange(classes):
#         if i == 0:
#             labels[(returns <= thresholdsMax[i])] = i

#         elif i == (classes-1):
#             labels[(returns >= thresholdsMin[i])] = i

#         else:
#             labels[(returns >= thresholdsMin[i]) & (returns<=thresholdsMax[i])] = i  
            
#     return labels

n_classes = 5
# load from imported extract_labels()
labels = extract_labels(data = candles, classes = n_classes, group_style = 'equal')
labels

(array([0., 4., 0., 1., 0., 4., 4., 1., 0., 3., 4., 2., 1., 0., 0., 0., 3.,
        2., 3., 4., 1., 0., 4., 0., 3., 3., 1., 0., 3., 3., 0., 3., 3., 1.,
        4., 4., 4., 4., 1., 2., 3., 2., 4., 4., 2., 2., 0., 3., 0., 3., 2.,
        1., 1., 0., 2., 2., 3., 0., 4., 2., 4., 3., 3., 2., 3., 2., 2., 3.,
        1., 1., 1., 4., 1., 1., 4., 1., 1., 0., 3., 1., 2., 3., 3., 0., 1.,
        2., 4., 1., 4., 1., 2., 1., 4., 3., 3., 2., 1., 2., 2., 1., 3., 0.,
        0., 4., 0., 3., 4., 3., 2., 3., 0., 1., 0., 1., 2., 1., 3., 1., 1.,
        1., 3., 3., 1., 3., 3., 0., 4., 2., 0., 1., 1., 1., 3., 4., 1., 0.,
        2., 2., 2., 4., 3., 1., 2., 2., 3., 1., 4., 1., 0., 2., 1., 1., 2.,
        2., 4., 1., 0., 1., 2., 4., 2., 0., 4., 4., 1., 1., 1., 2., 1., 3.,
        3., 1., 3., 1., 3., 0., 1., 3., 2., 2., 2., 3., 0., 3., 1., 1., 2.,
        0., 0., 2., 1., 0., 3., 4., 2., 1., 4., 0., 3., 3., 4., 3., 1., 4.,
        2., 1., 1., 0., 1., 1., 4., 4., 3., 3., 0., 3., 2., 3., 0., 2., 4.,
        2., 

In [17]:
# to transpose or not
%timeit ((candles.T[-1][1:]/candles.T[-1][0:-1])-1)*100
%timeit ((candles[1:, -1] / candles[:-1, -1])-1)*100
t1 = ((candles.T[-1][1:]/candles.T[-1][0:-1])-1)*100
t2 = ((candles[1:, -1] / candles[:-1, -1])-1)*100
sum(t1 == t2), len(t1), len(t2)

12.9 µs ± 850 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
11.6 µs ± 304 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


(779, 779, 779)

### The following is old down to "Align ..."

In [18]:
returns = ((candles.T[-1][1:]/candles.T[-1][0:-1])-1)*100
# Not tested
classes = 5
labels = np.zeros(returns.shape[0])#-featureWindow
thresholdsMin = [np.array_split(np.sort(returns),classes)[i].min() for i in np.arange(classes)]
thresholdsMax = [np.array_split(np.sort(returns),classes)[i].max() for i in np.arange(classes)]

In [19]:
# Not tested
for i in np.arange(classes):
    
    if i == 0:
        
        labels[(returns <= thresholdsMax[i])] = i
    
    elif i == (classes-1):
        
        labels[(returns >= thresholdsMin[i])] = i
    
    else:
        
        labels[(returns >= thresholdsMin[i])&(returns<=thresholdsMax[i])] = i

In [24]:
labels

array([0., 4., 0., 1., 0., 4., 4., 1., 0., 3., 4., 2., 1., 0., 0., 0., 3.,
       2., 3., 4., 1., 0., 4., 0., 3., 3., 1., 0., 3., 3., 0., 3., 3., 1.,
       4., 4., 4., 4., 1., 2., 3., 2., 4., 4., 2., 2., 0., 3., 0., 3., 2.,
       1., 1., 0., 2., 2., 3., 0., 4., 2., 4., 3., 3., 2., 3., 2., 2., 3.,
       1., 1., 1., 4., 1., 1., 4., 1., 1., 0., 3., 1., 2., 3., 3., 0., 1.,
       2., 4., 1., 4., 1., 2., 1., 4., 3., 3., 2., 1., 2., 2., 1., 3., 0.,
       0., 4., 0., 3., 4., 3., 2., 3., 0., 1., 0., 1., 2., 1., 3., 1., 1.,
       1., 3., 3., 1., 3., 3., 0., 4., 2., 0., 1., 1., 1., 3., 4., 1., 0.,
       2., 2., 2., 4., 3., 1., 2., 2., 3., 1., 4., 1., 0., 2., 1., 1., 2.,
       2., 4., 1., 0., 1., 2., 4., 2., 0., 4., 4., 1., 1., 1., 2., 1., 3.,
       3., 1., 3., 1., 3., 0., 1., 3., 2., 2., 2., 3., 0., 3., 1., 1., 2.,
       0., 0., 2., 1., 0., 3., 4., 2., 1., 4., 0., 3., 3., 4., 3., 1., 4.,
       2., 1., 1., 0., 1., 1., 4., 4., 3., 3., 0., 3., 2., 3., 0., 2., 4.,
       2., 1., 2., 0., 4.

In [25]:
features.shape

(780, 12)

In [26]:
labels.shape

(779,)

In [27]:
features.loc[0:50]

,stok_lag0,rsi_lag0,stok_lag1,rsi_lag1,stok_lag2,rsi_lag2,stok_lag3,rsi_lag3,stok_lag4,rsi_lag4,stok_lag5,rsi_lag5
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,75.088920,50.096776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,66.831334,47.132255,75.088920,50.096776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Align features and labels

In [28]:
# SAFE_BURN_IN = False # True for restarting burn-in each day, False for continuing from yesterday's close (DATA_SAMPLE = 'stable' requires SAFE_BURN_IN = True )

# if (DATA_SAMPLE == 'stable') & (not SAFE_BURN_IN):
#     raise ValueError(f'DATA_SAMPLE = "{DATA_SAMPLE}" & SAFE_BURN_IN = "{SAFE_BURN_IN}" not allowed. Burn-in needs a restart with DATA_SAMPLE = "{DATA_SAMPLE}" as we skip 30 minutes')

# # # postponing this for later
# # to do: allow burn-in to happen every day (we probably need to extract the MultiIndex from candles to do this in a nice vectorized way)
#     # 
# # if SAFE_BURN_IN:
    
# #     # for each day ...
    
# #     # get first index that has no NaNs (the sum checks for True across columns, we look for sum == 0 and where that is first True)
# #     burned_in_idx = np.where((np.sum(np.isnan(features.values), axis=1) == 0) == True)[0][0]
    
# #     # slice away the observations used for burn-in
# #     burned_in_features = features.iloc[burned_in_idx:, :] # features[burned_in_idx:] latter is sligthly faster but maybe not as precise
    
# #     # check if there are remaining NaNs are burn-in (means error)
# #     remaining_nans = np.where(np.isnan(burned_in_features.values))[0].size
# #     if remaining_nans > 0:
# #         raise ValueError('Had NaN in burned_in_features after burn-in')
        
# prediction_horizon = 1

# def align_features_and_labels(candles, prediction_horizon, features, n_feature_lags, n_classes,
#                               safe_burn_in = False, data_sample = 'full'):
    
#     if not safe_burn_in:
#         assert data_sample == 'full'
#         # we assume data_sample is full and that we can continue features from yesterday's values.
#         # that we have a single burn-in at the beginning and that's it

#         # get first index that has no NaNs (the sum checks for True across columns, we look for sum == 0 and where that is first True)
#         burned_in_idx = np.where((np.sum(np.isnan(features.values), axis=1) == 0) == True)[0][0]

#         # calculate end-point cut-off to match with labels
#         end_point_cut = max(prediction_horizon, n_feature_lags) + 1 #prediction_horizon + (FEATURE_LAGS - 2)

#         # slice away the observations used for burn-in (taking off 1 at the end to match with labels [slice off "prediction_horizon"])
#         burned_in_features = features.iloc[burned_in_idx : -end_point_cut, :].reset_index(drop=True) # features[burned_in_idx:] latter is sligthly faster but maybe not as precise

#         # slice away the burned-in indices from labels
#         labels, _, _ = extract_labels(data = candles[burned_in_idx+n_feature_lags:, :], 
#                                       classes = n_classes, group_style = 'equal')
#         # labels, returns, thresholds = extract_labels(data = candles[burned_in_idx + n_feature_lags : , :], 
#         #                                             classes = n_classes, group_style = 'equal')

#         # check if there are remaining NaNs are burn-in (means error)
#         remaining_nans = np.where(np.isnan(burned_in_features.values))[0].size
#         if remaining_nans > 0:
#             raise ValueError('Had NaN in burned_in_features after burn-in')   
            
#     return burned_in_features, labels # call the function as X, y = align_features_and_labels(.) if you like
        
# from imported function (see testing_preprocessing_features_and_labels.ipynb for thorough experimenting with all the cut-offs):    
X, y = align_features_and_labels(candles = candles, 
                                 prediction_horizon = 1, 
                                 features = features, 
                                 n_feature_lags = n_feature_lags, 
                                 n_classes = n_classes, # 5,
                                 safe_burn_in = False, 
                                 data_sample = 'full')

In [29]:
X.shape, y.shape

((761, 12), (761,))

## Split data into train and test set

In [30]:
#X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.1, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((684, 12), (77, 12), (684,), (77,))